In [14]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

# Runnable 协议

要支持链式调用，就必须实现 **Runnable** 协议。 
这是一个标准接口，可以轻松定义自定义链并以标准方式调用它们。 

标准接口包括：

- invoke：在输入上调用链
- batch：批量执行invoke
- stream：通过生成器实现流返回

这些也有相应的异步方法：

- ainvoke
- abatch
- astream
- astream_log：打印中间步骤
- astream_events：实现事件流（在 langchain-core 0.1.14 中引入）

不同组件的输入输出：

| 组件名称    | 输入类型   | 输出类型     |
|-----------|------------|-------------|
| Prompt模板    | 提示语和字典 | PromptValue |
| Retriever | 字符串 | 文档列表 |
| Tool      | 字符串或字典（依赖于工具的实现） | 依赖于工具的实现 |
| ChatModel | 字符串，对话消息或PrompValue | ChatMessage |
| LLM       | 字符串，对话消息或PrompValue | 字符串 |
| OutputParser | LLM 或 ChatModel 的输出 | 依赖于解析器的实现 |


# 输入输出schema

**chain**的输入输出都使用**pydantic**来检验**schema**是否合规。

In [15]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
chain = prompt | model

## Runnable的schema

In [10]:
prompt.input_schema()

PromptInput(topic=None)

In [2]:
prompt.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [ ]:
prompt.output_schema.schema()

## Chain的schema

### input_schema是链序列中第一个对象的

In [27]:
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [13]:
chain.first

ChatPromptTemplate(input_variables=['topic'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], template='tell me a joke about {topic}'))])

### output_schema是链序列中最后一个对象的

In [ ]:
chain.output_schema.schema()

In [12]:
chain.last

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x11475eb90>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x114798f40>, openai_api_key='sk-SctYLVDdKoqPLXoKlLEMjLqHG2BhuOtQgjmMWvrTDqlKJeqz', openai_proxy='')

# 接口调用

## invoke

定制 Runnable 时**必须重载**这个接口。

In [ ]:
chain.invoke({"topic": "bears"})

## ainvoke

缺省实现是在 langchain 内异步调用 invoke 接口；如果大模型有异步接口**应当重载**。

In [16]:
await chain.ainvoke({"topic": "bears"})

AIMessage(content='Why did the bear bring a flashlight to the party? \n\nBecause he heard it was going to be a "beary" good time!')

## batch

缺省实现是批量调用 invoke 接口；一般**不需要重载**。

In [19]:
chain.batch([{"topic": "bears"}, {"topic": "programmer"}])

[AIMessage(content="Why did the bear break up with his girlfriend? \n\nBecause he couldn't bear the distance between them!"),
 AIMessage(content='Why do programmers prefer dark mode? Because the light attracts too many bugs!')]

## abatch

缺省实现是批量调用 ainvoke 接口；一般**不需要重载**。

In [ ]:
await chain.abatch([{"topic": "bears"}, {"topic": "programmer"}])

## stream

缺省实现是调用 invoke 接口；一般都**需要重载**，否则就退化为 invoke 调用。

In [37]:
for s in chain.stream({"topic": "bears"}):
    print(s.content, end="", flush=True)

Sure! Here's a bear joke for you:

Why don't bears use cellphones?

Because they already have paws for "bear-y" good reception!

## astream

缺省实现是调用 ainvoke 接口；一般都**需要重载**，否则就退化为 ainvoke 调用。<br>
**值得注意**的是：即使实现了 stream 函数， asteram 的缺省实现也会调用 ainvoke（可能最后是间接调用 invoke ）

In [28]:
async for chunk in chain.astream({"topic": "bears"}):
    print(chunk.content, end="", flush=True)

Why did the bear bring a flashlight to the party?
Because he heard it was going to be a "beary" dark night!

## astream_events

调用 astream_log 实现，当前仅支持 v1 版本；一般**不考虑重载**。

In [44]:
async for chunk in chain.astream_events({"topic": "bears"}, version="v1", include_tags=['seq:step:2']):
    if 'chunk' in chunk['data']:
        print(chunk['data'], flush=True)

{'chunk': AIMessageChunk(content='')}
{'chunk': AIMessageChunk(content='Why')}
{'chunk': AIMessageChunk(content=' don')}
{'chunk': AIMessageChunk(content="'t")}
{'chunk': AIMessageChunk(content=' bears')}
{'chunk': AIMessageChunk(content=' wear')}
{'chunk': AIMessageChunk(content=' shoes')}
{'chunk': AIMessageChunk(content='?\n\n')}
{'chunk': AIMessageChunk(content='Because')}
{'chunk': AIMessageChunk(content=' they')}
{'chunk': AIMessageChunk(content=' have')}
{'chunk': AIMessageChunk(content=' bear')}
{'chunk': AIMessageChunk(content=' feet')}
{'chunk': AIMessageChunk(content='!')}
{'chunk': AIMessageChunk(content='')}


## astream_log

调用 stream 实现；一般**不考虑重载**。

In [43]:
async for s in chain.astream_log({"topic": "bears"}):
    print(s)

RunLogPatch({'op': 'replace',
  'path': '',
  'value': {'final_output': None,
            'id': 'a7f3478b-d30e-4717-a121-96dc988d2fb2',
            'logs': {},
            'name': 'RunnableSequence',
            'streamed_output': [],
            'type': 'chain'}})
RunLogPatch({'op': 'add',
  'path': '/logs/ChatPromptTemplate',
  'value': {'end_time': None,
            'final_output': None,
            'id': 'e0bcca54-4521-4a16-a8e3-1154e1c815a1',
            'metadata': {},
            'name': 'ChatPromptTemplate',
            'start_time': '2024-02-21T14:56:20.635+00:00',
            'streamed_output': [],
            'streamed_output_str': [],
            'tags': ['seq:step:1'],
            'type': 'prompt'}})
RunLogPatch({'op': 'add',
  'path': '/logs/ChatPromptTemplate/final_output',
  'value': ChatPromptValue(messages=[HumanMessage(content='tell me a joke about bears')])},
 {'op': 'add',
  'path': '/logs/ChatPromptTemplate/end_time',
  'value': '2024-02-21T14:56:20.636+00:00'})
R